### ScanEnergy 
Scanデータでのエネルギーと各タイプ・Rssとの関係性   
特に高エネルギー（__10keV__）以上のものを注目してみる

depressionアリの場合となしの場合でも比較    
（作業予想として）depression無しの場合のfluxは（典型的な）Neutral Sheetタイプのfluxの特徴に一致するはず...  
こっちは低エネルギーの特徴も見ていきたい    

単純にdepression領域内のflux（ない場合には，磁気赤道±5分の範囲）の平均を計算して比較してみる...?

例えば...
* 低エネルギーがどのような場合でも同じようにあるならNSが原因？太陽風によっては内側に来るなどもきいてくるはず...
* 高エネルギーが近傍のみにあるとring current的であると言えそう？外にもあるならring currentが外側にもできている？
  
みたいな結果が出たらうれしい

In [1]:
import MDUS
from MDUS import Analysis

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

In [ ]:
# データ格納の準備
result = pd.DataFrame()
orbits = []
datas = {}
for i in MDUS.EQTAB:
    if i >= 0.1:
        datas[i] = []
types = []
xs = []
ys = []
zs = []
rsss = []

nodata = []

# データ作成
for orbit in MDUS.orbit.index:
    # 計算する起動の設定
    # とりあえず，500番目までの計算
    # if orbit > 500:
    #     break
    
    # 軌道番号の表示
    print("\033[31m orbit \033[0m", orbit)

    # 各軌道データの取得
    data = MDUS.ScanData()
    data.Input(orbit=orbit)

    # scanデータがあるかないか
    if data.value is None or len(data.value[MDUS.EQTAB[-1]].values) == 0 or orbit in nodata:
        print("No data...")
        continue

    # 軌道データの取得
    data.DataIntegration()
    data.MLAT()

    # depressionか否かの判定
    if orbit in MDUS.dip.index:
        dt = np.abs(data.value["MLAT"]).idxmin()
        ds = MDUS.dip.query('index == @orbit')['start'].values[0]
        de = MDUS.dip.query('index == @orbit')['end'].values[0]
        diptype = MDUS.diptype.query('index == @orbit')['type'].values[0]
    else:
        dt = np.abs(data.value["MLAT"]).idxmin()
        ds = dt - pd.Timedelta(minutes=5)
        de = dt + pd.Timedelta(minutes=5)
        diptype = "none"

    # データの計算
    xs.append(data.value.query('index == @dt')["X_MSO"].values[0])
    ys.append(data.value.query('index == @dt')["Y_MSO"].values[0])
    zs.append(data.value.query('index == @dt')["Z_MSO"].values[0])
    orbits.append(orbit)
    types.append(diptype)
    rsss.append(MDUS.Mercury.query('index == @orbit')['Rss'].values[0])

    for energy in MDUS.EQTAB:
        if energy >= 0.1:
            datas[energy].append(np.mean(data.value.query('@ds <= index <= @de')[energy].values))
    print("Done !")
# データの格納
result['orbit'] = orbits
for key in datas.keys():
    result[key] = datas[key]
result['X'] = xs
result['Y'] = ys
result['Z'] = zs
result['type'] = types
result['Rss'] = rsss
result = result.set_index('orbit')
result.to_pickle("./data/other/ScanAverage.pkl")

In [2]:
result = pd.read_pickle("./data/other/ScanAverage.pkl")
datas = {}
for i in MDUS.EQTAB:
    if i >= 0.1:
        datas[i] = []
result

,0.1004,0.1107,0.1219,0.134,0.1478,0.1627,0.1789,0.197,0.217,0.2388,...,13.5774,14.9482,16.4578,18.1198,19.9492,X,Y,Z,type,Rss
orbit,,,,,,,,,,,,,,,,,,,,,
12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.752777,-1.028571,0.221939,none,1.628600
13,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.800408,-0.983537,0.234643,none,1.365027
14,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.868918,-0.980401,0.166869,none,1.622557
15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-0.912552,-0.931622,0.179677,none,1.485355
16,9.010000e+05,1.360111e+06,1.718222e+06,6.710000e+05,1.821889e+06,1.283000e+06,8.308889e+05,7.544444e+05,8.193333e+05,6.188889e+05,...,2.577867e+05,1.983811e+05,1.458311e+05,7.906467e+04,3.880222e+04,-0.952689,-0.881361,0.192979,none,1.371448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4100,0.000000e+00,1.969825e+06,8.889474e+05,1.619649e+06,3.679123e+06,1.985789e+06,1.202456e+06,1.638947e+06,4.936842e+05,4.473684e+05,...,4.179474e+04,1.204211e+04,1.380351e+04,5.901754e+03,1.505263e+04,-2.137280,-1.304290,0.198923,none,1.489417
4101,3.080031e+06,4.035691e+06,3.027041e+06,3.451701e+06,2.124165e+06,3.080021e+06,1.858577e+06,2.973763e+06,2.495711e+06,2.549021e+06,...,3.132835e+06,3.239082e+06,2.124072e+06,3.079804e+06,3.504990e+06,0.970188,0.593886,0.200135,none,1.321999
4102,0.000000e+00,0.000000e+00,1.777895e+06,0.000000e+00,7.357895e+05,1.323860e+06,1.803684e+06,1.092632e+06,1.481053e+06,1.789474e+06,...,8.358947e+04,6.421930e+04,6.902632e+04,5.017895e+04,3.763158e+04,-2.216934,-1.162198,0.198723,none,1.262111


In [18]:
for energy in datas.keys():
    fig, ax = plt.subplots()
    ax.set_title(str(energy) + ' keV/Q')
    tmp = result.query('type == "dip"')[energy]
    hoge = result.query('type == "dip"')['Rss']
    ax.scatter(hoge, tmp, c='r', label='dip/' + '{:.3e}'.format(np.mean(tmp)) + '/' + '{:.3e}'.format(np.std(tmp)) ,zorder=3,alpha=0.7)

    tmp = result.query('type == "ns"')[energy]
    hoge = result.query('type == "ns"')['Rss']
    ax.scatter(hoge, tmp, c='b', label='ns/' + '{:.3e}'.format(np.mean(tmp)) + '/' + '{:.3e}'.format(np.std(tmp)),zorder=2,alpha=0.7)

    tmp = result.query('type == "none"')[energy]
    hoge = result.query('type == "none"')['Rss']
    ax.scatter(hoge, tmp, c='lightgrey', label='none/' + '{:.3e}'.format(np.mean(tmp)) + '/' + '{:.3e}'.format(np.std(tmp)),zorder=1)
    # ax.set_ylim(1e3,1e9)
    ax.set_yscale('log')
    ax.set_xlabel('Rss')
    ax.set_ylabel('Flux')
    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig('./figure/flux/' + str(energy) + '_flux_rss.png',dpi=400)
    plt.close()

In [24]:
cm = plt.cm.get_cmap('RdYlBu')
cm = 'jet'
for energy in datas.keys():
    fig, ax = plt.subplots()
    type = 'dip'
    ax.set_title(type + '/' + str(energy) + ' keV/Q')
    ene = result.query('type == @type')[energy]
    rss = result.query('type == @type')['Rss']
    x = result.query('type == @type')['X']
    y = result.query('type == @type')['Y']
    z = result.query('type == @type')['Z']
    r = np.sqrt(x**2 + y**2 + z**2)
    m = ax.scatter(r,ene,c=rss,cmap=cm, label='dip/' + '{:.3e}'.format(np.mean(ene)) + '/' + '{:.3e}'.format(np.std(ene)) ,zorder=3)
    fig.colorbar(m, ax=ax)
    ax.set_xlabel('Range')
    ax.set_ylabel('Flux')
    plt.tight_layout()
    plt.savefig('./figure/flux/' + type + '_' + str(energy) + '_flux_rss_x.png',dpi=400)
    plt.close()

    fig, ax = plt.subplots()
    type = 'ns'
    ax.set_title(type + '/' + str(energy) + ' keV/Q')
    ene = result.query('type == @type')[energy]
    rss = result.query('type == @type')['Rss']
    x = result.query('type == @type')['X']
    y = result.query('type == @type')['Y']
    z = result.query('type == @type')['Z']
    r = np.sqrt(x**2 + y**2 + z**2)
    m = ax.scatter(r,ene,c=rss,cmap=cm, label='dip/' + '{:.3e}'.format(np.mean(ene)) + '/' + '{:.3e}'.format(np.std(ene)) ,zorder=3)
    fig.colorbar(m, ax=ax)
    ax.set_xlabel('Range')
    ax.set_ylabel('Flux')
    plt.tight_layout()
    plt.savefig('./figure/flux/' + type + '_' + str(energy) + '_flux_rss_x.png',dpi=400)
    plt.close()

    fig, ax = plt.subplots()
    type = 'none'
    ax.set_title(type + '/' + str(energy) + ' keV/Q')
    ene = result.query('type == @type')[energy]
    rss = result.query('type == @type')['Rss']
    x = result.query('type == @type')['X']
    y = result.query('type == @type')['Y']
    z = result.query('type == @type')['Z']
    r = np.sqrt(x**2 + y**2 + z**2)
    m = ax.scatter(r,ene,c=rss,cmap=cm, label='dip/' + '{:.3e}'.format(np.mean(ene)) + '/' + '{:.3e}'.format(np.std(ene)) ,zorder=3)
    fig.colorbar(m, ax=ax)
    ax.set_xlabel('Range')
    ax.set_ylabel('Flux')
    plt.tight_layout()
    plt.savefig('./figure/flux/' + type + '_' + str(energy) + '_flux_rss_x.png',dpi=400)
    plt.close()

/tmp/ipykernel_81634/3776992325.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('RdYlBu')
